In [18]:
import bw2io as bi
import bw2calc as bc
import bw2data as bd
import pandas as pd
import numpy as np

In [19]:
PROJECT_NAME = "DdS_textile"
TAR_PATH     = '/etc/data/ecoinvent-3.10-cutoff-bw25.tar.gz'

# If the project already exists, just switch to it,
# otherwise restore from archive and then switch.
if PROJECT_NAME in bd.projects:
    bd.projects.set_current(PROJECT_NAME)
else:
    bi.restore_project_directory(
        fp=TAR_PATH,
        project_name=PROJECT_NAME,
    )
    bd.projects.set_current(PROJECT_NAME)


In [20]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

In Life Cycle Assessment (LCA), the "avoided burden" approach gives credit to a system when it produces outputs that replace products made elsewhere. For example, if a waste management system recovers textiles and turns them into recycled fibers that can replace virgin textiles, the environmental impacts of producing the virgin textiles are subtracted from the system's total footprint. This reflects the environmental benefit of recycling by showing how the system helps avoid emissions and resource use that would have occurred in traditional textile production.

In [21]:
def get_unique_node(name, location, unit, reference_product, database="ecoinvent-3.10-cutoff"):
    matches = [
        act for act in bd.Database(database)
        if act.get('name') == name
        and act.get('location') == location
        and act.get('unit') == unit
        and act.get('reference product') == reference_product
    ]
    
    if len(matches) == 0:
        raise ValueError(f"No match found for: {name} - {location} - {unit} - {reference_product}")
    elif len(matches) == 1:
        return matches[0]
    elif len(matches) > 1:
        raise ValueError(f"Multiple matches found for: {name} - {location} - {unit} - {reference_product}")

avoided_fiber_production_processes = {
    "acrylic": get_unique_node("acrylic acid production", "RER", "kilogram", "acrylic acid"), 
    "cotton": get_unique_node("fibre production, cotton, ginning", "RoW", "kilogram", "fibre, cotton"),
    "polyester": get_unique_node("polyester fibre production, finished", "RoW", "kilogram", "fibre, polyester"),
    "viscose": get_unique_node("fibre production, viscose", "GLO", "kilogram", "fibre, viscose"),
    "silk": get_unique_node("market for fibre, silk, short", "GLO", "kilogram", "fibre, silk, short"),
    "nylon": get_unique_node("nylon 6-6 production", "RoW", "kilogram", "nylon 6-6"),
    "wool": get_unique_node("sheep production, for wool", "US", "kilogram", "sheep fleece in the grease"),
}
avoided_fiber_production_processes

{'acrylic': 'acrylic acid production' (kilogram, RER, None),
 'cotton': 'fibre production, cotton, ginning' (kilogram, RoW, None),
 'polyester': 'polyester fibre production, finished' (kilogram, RoW, None),
 'viscose': 'fibre production, viscose' (kilogram, GLO, None),
 'silk': 'market for fibre, silk, short' (kilogram, GLO, None),
 'nylon': 'nylon 6-6 production' (kilogram, RoW, None),
 'wool': 'sheep production, for wool' (kilogram, US, None)}

# Basic Combustion Model Summary for Common Textile Fibers

| **Fiber**           | **Chemical Nature**         | **Repeat Unit Formula** | **Molar Mass (g/mol)** | **Notable Combustion Products**       | **HHV (MJ/kg)** |
|---------------------|-----------------------------|--------------------------|------------------------|----------------------------------------|-----------------|
| **Cotton**          | Natural cellulose           | C₆H₁₀O₅                 | 162                    | CO₂, H₂O                               | ~17             |
| **Wool**            | Natural protein (keratin)   | C₂H₅NO + trace S         | ~75                   | CO₂, H₂O, NOₓ, SO₂                      | ~23             |
| **Silk**            | Natural protein (fibroin)   | C₁₅H₂₀N₅O₆              | 365                    | CO₂, H₂O, NOₓ                           | ~20             |
| **Viscose**         | Regenerated cellulose       | C₆H₁₀O₅                 | 162                    | CO₂, H₂O                               | ~16             |
| **Polyester (PET)** | Synthetic polymer           | C₁₀H₈O₄                 | 192                    | CO₂, H₂O, CO, soot                      | ~24             |
| **Polyamide/Nylon** | Synthetic polymer           | C₁₂H₂₂N₂O₂              | 226                    | CO₂, H₂O, NOₓ                           | ~30             |
| **Acrylic (PAN)**   | Synthetic polymer           | C₃H₃N                   | 53                     | CO₂, H₂O, HCN                           | ~28             |
                      |

> Notes:
> - All combustion reactions assume **complete oxidation**.
> - **Nitrogen-containing** fibers release NOₓ (e.g. wool, silk, nylon).
> - **Sulfur-containing** fibers (e.g. wool) produce SO₂.
> - **Acrylic** releases **hydrogen cyanide (HCN)** upon burning.
> - Higher heating value approximated based on literature values / chemical composition


In [22]:
from dataclasses import dataclass
from typing import Dict
import pprint
import bw2data as bd

@dataclass
class Fiber:
    name: str
    formula: Dict[str, int]  # Elemental composition per repeat unit
    molar_mass: float        # g/mol
    byproducts: Dict[str, int]  # Special combustion byproducts like HCN, SO2, NOx

# Define fibers
fibers = [
    Fiber("Cotton", {"C":6, "H":10, "O":5}, 162, {}),
    Fiber("Wool", {"C":2, "H":5, "O":1, "N":1, "S":0.05}, 75, {"SO2": 0.05, "NOx": 1}),
    Fiber("Silk", {"C":15, "H":20, "O":6, "N":5}, 365, {"NOx": 5}),
    Fiber("Viscose", {"C":6, "H":10, "O":5}, 162, {}),
    Fiber("Polyester", {"C":10, "H":8, "O":4}, 192, {}),
    Fiber("Polyamide/Nylon", {"C":12, "H":22, "O":2, "N":2}, 226, {"NOx": 2}),
    Fiber("Acrylic", {"C":3, "H":3, "N":1}, 53, {"HCN": 1}),
]

# Molar masses
atomic_weights = {
    "C": 12.01,
    "H": 1.008,
    "O": 16.00,
    "N": 14.01,
    "S": 32.06,
    "CO2": 44.01,
    "H2O": 18.02,
    "SO2": 64.06,
    "NOx": 46.00,  # Approx for NO2
    "HCN": 27.03,
}

def combustion_products(fiber: Fiber):
    moles = 1000 / fiber.molar_mass  # 1 kg of fiber
    products = {}

    # CO2: 1 per C atom
    if "C" in fiber.formula:
        products["CO2"] = moles * fiber.formula["C"]

    # H2O: 0.5 per H atom
    if "H" in fiber.formula:
        products["H2O"] = moles * fiber.formula["H"] / 2

    # NOx, SO2, HCN etc.
    for gas, count in fiber.byproducts.items():
        products[gas] = moles * count

    # Convert to mass (g)
    products_mass = {gas: round(mol * atomic_weights[gas] / 1000, 3) for gas, mol in products.items()}  # in kg

    return products_mass

# Display results
for fiber in fibers:
    mass_output = combustion_products(fiber)
    print(f"\n Combustion products from 1 kg of {fiber.name}:")
    pprint.pprint(mass_output)



 Combustion products from 1 kg of Cotton:
{'CO2': 1.63, 'H2O': 0.556}

 Combustion products from 1 kg of Wool:
{'CO2': 1.174, 'H2O': 0.601, 'NOx': 0.613, 'SO2': 0.043}

 Combustion products from 1 kg of Silk:
{'CO2': 1.809, 'H2O': 0.494, 'NOx': 0.63}

 Combustion products from 1 kg of Viscose:
{'CO2': 1.63, 'H2O': 0.556}

 Combustion products from 1 kg of Polyester:
{'CO2': 2.292, 'H2O': 0.375}

 Combustion products from 1 kg of Polyamide/Nylon:
{'CO2': 2.337, 'H2O': 0.877, 'NOx': 0.407}

 Combustion products from 1 kg of Acrylic:
{'CO2': 2.491, 'H2O': 0.51, 'HCN': 0.51}


In [23]:
def get_biosphere_flow(name, unit=None, categories=None, database="ecoinvent-3.10-biosphere"):
    matches = [
        flow for flow in bd.Database(database)
        if flow.get('name') == name
        and (unit is None or flow.get('unit') == unit)
        and (categories is None or flow.get('categories') == categories)
    ]

    if len(matches) == 0:
        raise ValueError(f" No biosphere flow found for: {name} - {unit} - {categories}")
    elif len(matches) == 1:
        return matches[0]
    else:
        print(f"⚠️ Multiple matches found for: {name}")
        for i, match in enumerate(matches):
            print(f"  Match {i+1}: {match['name']} | {match.get('unit')} | {match.get('categories')}")
        raise ValueError(" Too many matches. Refine your query.")
    
element_flows = {
    "H2O": get_biosphere_flow("Water, in air", "cubic meter", ("natural resource", "in air")),
    "CO2": get_biosphere_flow("Carbon dioxide, fossil", "kilogram", ("air",)),
    "NOx": get_biosphere_flow("Nitrogen oxides", "kilogram", ("air",)),
    "SO2": get_biosphere_flow("Sulfur dioxide", "kilogram", ("air",)),
    "CO": get_biosphere_flow("Carbon monoxide, fossil", "kilogram", ("air",)),
    "HCN": get_biosphere_flow("Hydrogen cyanide", "kilogram", ("air",)),
}
element_flows

{'H2O': 'Water, in air' (cubic meter, None, ('natural resource', 'in air')),
 'CO2': 'Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'NOx': 'Nitrogen oxides' (kilogram, None, ('air',)),
 'SO2': 'Sulfur dioxide' (kilogram, None, ('air',)),
 'CO': 'Carbon monoxide, fossil' (kilogram, None, ('air',)),
 'HCN': 'Hydrogen cyanide' (kilogram, None, ('air',))}

In [24]:
fiber_db = bd.Database("fiber_combustion")
activity_data = {}

for fiber in fibers:
    outputs = combustion_products(fiber)

    exchanges = []
    for elem, amount in outputs.items():
        flow = element_flows.get(elem)
        if not flow:
            print(f"⚠️ {elem} not found — skipping in {fiber.name}")
            continue
        exchanges.append({
            'input': flow.key,
            'amount': amount,
            'type': 'biosphere',
            'unit': flow['unit'],
            'name': flow['name'],
        })

    exchanges.append({
        'input': ("fiber_combustion", f"{fiber.name.lower()}_combustion"),
        'amount': 1.0,
        'type': 'production',
        'unit': "kilogram",
        'name': f"Combustion of {fiber.name}",
    })

    activity_data[("fiber_combustion", f"{fiber.name.lower()}_combustion")] = {
        'name': f"Combustion of {fiber.name}",
        'unit': "kilogram",
        'exchanges': exchanges,
    }

fiber_db.write(activity_data)
print("✅ Regenerated all activities with updated flows.")

12:20:28+0800 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 7/7 [00:00<?, ?it/s]


12:20:28+0800 [info     ] Vacuuming database            
✅ Regenerated all activities with updated flows.


In [ ]:
fiber_combustion = {
    fiber.name: bd.get_activity(("fiber_combustion", f"{fiber.name.lower()}_combustion"))
    for fiber in fibers
}

methods = {
    "acidification": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'acidification no LT', 'accumulated exceedance (AE) no LT'
    ),
    "climate change": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'climate change no LT', 'global warming potential (GWP100) no LT'
    ),
    "ecotoxicity": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'ecotoxicity: freshwater no LT', 'comparative toxic unit for ecosystems (CTUe) no LT'
    ),
    "fossil depletion": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'energy resources: non-renewable no LT', 'abiotic depletion potential (ADP): fossil fuels no LT'
    ),
    "eutrophication freshwater": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'eutrophication: freshwater no LT', 'fraction of nutrients reaching freshwater end compartment (P) no LT'
    ),
    "human toxicity carcinogenic": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'human toxicity: carcinogenic no LT', 'comparative toxic unit for human (CTUh) no LT'
    ),
    "human toxicity non-carcinogenic": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'human toxicity: non-carcinogenic no LT', 'comparative toxic unit for human (CTUh) no LT'
    ),
    "ionising radiation": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'ionising radiation: human health no LT', 'human exposure efficiency relative to u235 no LT'
    ),
    "land use": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'land use no LT', 'soil quality index no LT'
    ),
    "metal depletion": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'material resources: metals/minerals no LT', 'abiotic depletion potential (ADP): elements (ultimate reserves) no LT'
    ),
    "ozone depletion": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'ozone depletion no LT', 'ozone depletion potential (ODP) no LT'
    ),
    "particulate matter": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'particulate matter formation no LT', 'impact on human health no LT'
    ),
    "smog formation": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'photochemical oxidant formation: human health no LT', 'tropospheric ozone concentration increase no LT'
    ),
    "water use": (
        'ecoinvent-3.10', 'EF v3.1 no LT',
        'water use no LT', 'user deprivation potential (deprivation-weighted water consumption) no LT'
    ),
}

results = np.zeros((len(fiber_combustion), len(methods)))
products = list(fiber_combustion.values())

for i, product in enumerate(products):
    for j, method in enumerate(methods.values()):
        lca = bc.LCA({product: 1}, method=method)
        lca.lci()
        lca.lcia()
        results[i, j] = lca.score

df_results = pd.DataFrame(
    results,
    index=[p['name'] for p in products],
    columns=methods.keys()
)

df_results = df_results.loc[:, (df_results != 0).any(axis=0)]
display(df_results)


c:\Users\HP\Desktop\Repos\textileanatomy_dds\venv\Lib\site-packages\bw2calc\lca.py:247: UserWarning: All values in characterization matrix are zero
  warnings.warn("All values in characterization matrix are zero")


,acidification,climate change,ecotoxicity,human toxicity non-carcinogenic,particulate matter,smog formation
Combustion of Cotton,0.00000,-1.630,0.000000,0.000000,0.000000e+00,0.000000
Combustion of Wool,-0.50995,-1.174,0.000000,0.000000,-1.324800e-06,-0.616487
Combustion of Silk,-0.46620,-1.809,0.000000,0.000000,-1.008000e-06,-0.630000
Combustion of Viscose,0.00000,-1.630,0.000000,0.000000,0.000000e+00,0.000000
Combustion of Polyester,0.00000,-2.292,0.000000,0.000000,0.000000e+00,0.000000
Combustion of Polyamide/Nylon,-0.30118,-2.337,0.000000,0.000000,-6.512000e-07,-0.407000
Combustion of Acrylic,0.00000,-2.491,-1199.774978,-0.000002,0.000000e+00,0.000000


In [27]:
# for m in bd.methods:
#     if "climate" in m[1].lower():
#         print(m)